# Estructura del Modelo Seq2Seq Base

En este cuaderno, instanciaremos las clases Encoder, Decoder y Seq2Seq definidas en `src/model.py`. Definiremos los hiperparámetros y pasaremos un lote de datos a través del modelo para verificar que la estructura y las dimensiones son correctas antes de proceder con la implementación de la atención y el entrenamiento.

## model.py

In [20]:
# Pegamos las clases de model.py

import torch
import torch.nn as nn
import random

class Encoder(nn.Module):
    def __init__(self,input_dim, emb_dim, hidden_dim, n_layers, dropout, pad_idx):
        """
            Constructor del Encoder.
            Args: 
                input_dim(int): tamanio del vocabulario de entrada (fuente o sorce o src).
                emb_dim (int): Dimension de los embeddings.
                hidden_dim(int): dimension de la capa oculta del LSTM.
                n_layers (int): Numero de capas del LSTM
                dropout (float): Probabilidad de dropout
                pad_idx (idx): Indice del token de padding en el vocabulario
        """
        super().__init__() # Configuraciones internas de nn.Module en el Encoder
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_idx)

        self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers,
                           dropout=dropout if n_layers>1 else 0,
                           bidirectional=True, batch_first=True)
        
        self.fc_hidden = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc_cell = nn.Linear(hidden_dim * 2, hidden_dim)

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        """
            Procesa la secuencia fuente.
            Arg:
                src (Tensor): Secuencia de tokens de entrada [batch_size, src_len]
            Return:

        """
        embedded = self.dropout(self.embedding(src))

        outputs, (hidden, cell) = self.rnn(embedded)

        hidden = hidden.permute(1, 0, 2)
        
        hidden = hidden.reshape(hidden.size(0), self.n_layers, 2 * self.hidden_dim)

        hidden = hidden.permute(1, 0, 2)

        cell = cell.permute(1, 0, 2)
        cell = cell.reshape(cell.size(0), self.n_layers, 2 * self.hidden_dim)
        cell = cell.permute(1, 0, 2)

        hidden = torch.tanh(self.fc_hidden(hidden))
        cell = torch.tanh(self.fc_cell(cell))

        return outputs, hidden, cell

class Decoder(nn.Module):
    def __init__(self,output_dim, emb_dim, hidden_dim, n_layers, dropout, pad_idx):
        """
            Inicializador del Decoder.
            Args:
                output_dim(int): tamanio del vocabulario de salida.
                emb_dim(int): dimension de los embeddings.
                hidden_dim(int): dimension de la capa oculta del LSTM
                n_layers(int): Numero de capas del LSTM
                dropout (float): Probabilidad de dropout
                pad_idx: Indice del token de padding en el vocabulario
        """
        super().__init__() # Configuraciones internas del Module.nn en el Decoder

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=pad_idx)

        self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers,
                           dropout=dropout if n_layers > 1 else 0, batch_first=True)
        
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        """
            Procesa un paso de la decodificación:
            Arg:
                input(Tensor): Token de entrada actual [batch size]
                hidden(Tensor): Estado oculto del paso anterior [n_layers, batch_size, hidden_dim].
                cell(Tensor): Estado de la celda en el paso anterior  [n_layers, batch_size, hidden_dim].
            Return:
        """

        input = input.unsqueeze(1) # input = [batch size, 1]
        embedded = self.dropout(self.embedding(input)) # embedded = [batch size, 1, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        prediction = self.fc_out(output.squeeze(1))

        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        """
            Args:
                encoder(Encoder): instancia del encoder
                decoder(Decoder): instancia del decoder
                device(torch.device): cpu o cuda 
        """
        super().__init__() # Configuraciones internas de nn.Modules en Seq2Seq

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hidden_dim == decoder.hidden_dim, \
            "Las dimensiones ocultas del encoder y decoder deben de ser iguales"
        assert encoder.n_layers == decoder.n_layers, \
            "El encoder y decoder deben de tener el mismo numero de capas"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        """
            Procesa el par de secuencias fuente y objetivo.
            Args:
                src(Tensor): secuencia fuente [batch_size, src_len].
                trg(Tensor): secuencia target [batch_size, trg_len].
                teacher_forcing_ratio (float): Probabilidad de usar teacher forcing.
            
            Return:
                output(Tensor): predicciones del decoder.
        """
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        encoder_outputs, hidden, cell = self.encoder(src)

        input = trg[:, 0]

        for t in range(1, trg_len): # Predecimos a partir del segundo token

            output, hidden, cell = self.decoder(input, hidden, cell)

            # Guardamos las predicciones en el tensor de salida
            outputs[:, t, :] = output 

            # Decidir si usar teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio

            # Obtener el token predicho con mayor probabilidad
            top1 = output.argmax(1) 

            # Si es teacher forcing, usar el token real como siguiente input
            # Si no, usar el token predicho
            input = trg[:, t] if teacher_force else top1


        return outputs

## data_loader.py

In [11]:
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import torch

class SummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len_article, max_len_highlight, bos_token_id, eos_token_id):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len_article = max_len_article - 2  # Reservar espacio para BOS y EOS
        self.max_len_highlight = max_len_highlight - 2
        self.bos_token_id = bos_token_id
        self.eos_token_id = eos_token_id

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        article_text = self.dataframe.iloc[idx]['article']
        highlight_text = self.dataframe.iloc[idx]['highlights']

        # Tokenizar y truncar artículo
        self.tokenizer.enable_truncation(max_length=self.max_len_article)
        encoded_article = self.tokenizer.encode(article_text)
        article_token_ids = encoded_article.ids

        # Tokenizar y truncar resumen
        self.tokenizer.enable_truncation(max_length=self.max_len_highlight)
        encoded_highlight = self.tokenizer.encode(highlight_text)
        highlight_token_ids = encoded_highlight.ids

        # Añadir tokens BOS/EOS y convertir a tensor
        article_tensor = torch.cat(
            (torch.tensor([self.bos_token_id]),
             torch.tensor(article_token_ids, dtype=torch.long),
             torch.tensor([self.eos_token_id]))
        )

        highlight_tensor = torch.cat(
            (torch.tensor([self.bos_token_id]),
             torch.tensor(highlight_token_ids, dtype=torch.long),
             torch.tensor([self.eos_token_id]))
        )

        return article_tensor, highlight_tensor

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)

    src_batch_padded = pad_sequence(src_batch, batch_first=True, padding_value=1)  # 1 = PAD_IDX
    tgt_batch_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=1)

    return src_batch_padded, tgt_batch_padded

## Cargamos el tokenizer

In [12]:
import os
from tokenizers import Tokenizer

# Cargamos el tokenizer
TOKENIZER_DIR = "cnn_dailymail_bpe_tokenizer" 
TOKENIZER_PATH = os.path.join(TOKENIZER_DIR, "tokenizer.json")


tokenizer = Tokenizer.from_file(TOKENIZER_PATH)
print(f"Tokenizador cargado desde: {TOKENIZER_PATH}")
INPUT_DIM = tokenizer.get_vocab_size()
OUTPUT_DIM = tokenizer.get_vocab_size() 

Tokenizador cargado desde: cnn_dailymail_bpe_tokenizer\tokenizer.json


In [14]:
PAD_IDX = tokenizer.token_to_id("<pad>")
BOS_IDX = tokenizer.token_to_id("<bos>")
EOS_IDX = tokenizer.token_to_id("<eos>")
UNK_IDX = tokenizer.token_to_id("<unk>")

In [18]:
print(f"Tamaño del Vocabulario (INPUT_DIM/OUTPUT_DIM): {INPUT_DIM}")
print(f"Índice de OOV: {UNK_IDX}")
print(f"Índice de Padding: {PAD_IDX}")
print(f"Índice de Begin of seq.: {BOS_IDX}")
print(f"Índice de End of seq.: {EOS_IDX}")


Tamaño del Vocabulario (INPUT_DIM/OUTPUT_DIM): 30000
Índice de OOV: 0
Índice de Padding: 1
Índice de Begin of seq.: 2
Índice de End of seq.: 3


## Hiperparametros del modelo

In [13]:
ENC_EMB_DIM = 256  # Dimension Embedding Encoder 
DEC_EMB_DIM = 256  # Dimension Embedding Decoder
HID_DIM = 512      # Dimension Oculta LSTM
N_LAYERS = 2       # Numero de capas LSTM 
ENC_DROPOUT = 0.3  # Dropout Encoder 
DEC_DROPOUT = 0.3  # Dropout Decoder 

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Hiperparámetros definidos:")
print(f"  ENC_EMB_DIM: {ENC_EMB_DIM}")
print(f"  DEC_EMB_DIM: {DEC_EMB_DIM}")
print(f"  HID_DIM: {HID_DIM}")
print(f"  N_LAYERS: {N_LAYERS}")
print(f"  ENC_DROPOUT: {ENC_DROPOUT}")
print(f"  DEC_DROPOUT: {DEC_DROPOUT}")
print(f"  Device: {DEVICE}")

Hiperparámetros definidos:
  ENC_EMB_DIM: 256
  DEC_EMB_DIM: 256
  HID_DIM: 512
  N_LAYERS: 2
  ENC_DROPOUT: 0.3
  DEC_DROPOUT: 0.3
  Device: cpu


## Instanciar Hiperparametros del modelo

In [21]:
# Tratamos de instancias el Encoder y Decoder
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, PAD_IDX)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, PAD_IDX)

## Instanciar la clase Seq2Seq

In [22]:
model = Seq2Seq(enc, dec, DEVICE).to(DEVICE)
print("Instancia del modelo Seq2Seq creada y movida al device.")

# Función para contar parámetros
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model):,} parámetros entrenables.')



Instancia del modelo Seq2Seq creada y movida al device.
El modelo tiene 44,931,376 parámetros entrenables.


In [25]:
print("\nEstructura del Modelo:")
model


Estructura del Modelo:


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(30000, 256, padding_idx=1)
    (rnn): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
    (fc_hidden): Linear(in_features=1024, out_features=512, bias=True)
    (fc_cell): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(30000, 256, padding_idx=1)
    (rnn): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.3)
    (fc_out): Linear(in_features=512, out_features=30000, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

## Preparamos un Batch de datos para prueba

In [27]:
import pandas as pd
from torch.utils.data import DataLoader

MAX_TOKENS_ARTICLE = 1200
MAX_TOKENS_HIGHLIGHT = 130

train_df_filtered = pd.read_parquet("data/train_filtered.parquet") 
print("DataFrame de entrenamiento filtrado cargado.")

# Instanciamos el Dataset
temp_train_dataset = SummarizationDataset(
    train_df_filtered, tokenizer,
    MAX_TOKENS_ARTICLE, MAX_TOKENS_HIGHLIGHT, 
    BOS_IDX, EOS_IDX
)

# Instanciamos el Dataloader
BATCH_SIZE_TEST = 4 # Usamos un batch de 4 para probar
temp_train_dataloader = DataLoader(
    dataset=temp_train_dataset,
    batch_size=BATCH_SIZE_TEST,
    shuffle=False,
    collate_fn=collate_fn
)

# Obtenemos un batch
src_batch, trg_batch = next(iter(temp_train_dataloader))
print(f"\nPrimer batch obtenido: {BATCH_SIZE_TEST}")
print(f"  Shape src_batch: {src_batch.shape}")
print(f"  Shape trg_batch: {trg_batch.shape}")

DataFrame de entrenamiento filtrado cargado.

Primer batch obtenido: 4
  Shape src_batch: torch.Size([4, 923])
  Shape trg_batch: torch.Size([4, 68])


## Probamos el Forward Pass del Modelo

In [31]:
model.eval() # Poner el modelo en modo evaluación (desactiva dropout)

with torch.no_grad(): # sin gradientes
    # Mover datos al mismo dispositivo que el modelo
    src_batch = src_batch.to(DEVICE)
    trg_batch = trg_batch.to(DEVICE)

    print("\nEjecutando forward pass del modelo...")
    try:
        outputs = model(src_batch, trg_batch) # teacher_forcing_ratio=0 para probar sin él

        print("Forward pass completado sin errores.")
        print(f"  Shape de la salida (outputs): {outputs.shape}")

        expected_shape = (BATCH_SIZE_TEST, trg_batch.shape[1] , OUTPUT_DIM) 
        print(f"  Shape esperada: {expected_shape}")

        if outputs.shape == expected_shape:
            print("¡Las dimensiones de salida son correctas!")
        else:
            print("¡Advertencia! Las dimensiones de salida NO coinciden con las esperadas.")
            print(f"Verifica la implementación de Seq2Seq.forward y el uso de batch_first en el Decoder.")

    except Exception as e:
        print(f"\nError durante el forward pass: {e}")
        import traceback
        traceback.print_exc()


Ejecutando forward pass del modelo...
Forward pass completado sin errores.
  Shape de la salida (outputs): torch.Size([4, 68, 30000])
  Shape esperada: (4, 68, 30000)
¡Las dimensiones de salida son correctas!
